In [1]:
pip install sentence-transformers

  Using cached sentence_transformers-3.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.46.2-py3-none-any.whl.metadata (44 kB)
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached pillow-11.0.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja

# Set Environment Variables & API Keys

In [3]:
!pip install python-dotenv

In [4]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [5]:
import os 

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = "langchain-tutorial"
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

# Part 1: Overview

In [9]:
!pip install langchain_community langchain langchain-groq langchain_core

In [11]:
!pip install beautifulsoup4

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)


In [19]:
pip install faiss-cpu

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
    --------------------------------------- 0.3/14.9 MB ? eta -:--:--
    --------------------------------------- 0.3/14.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.9 MB 699.0 kB/s eta 0:00:21
   -- ------------------------------------- 0.8/14.9 MB 882.6 kB/s eta 0:00:16
   -- ------------------------------------- 0.8/14.9 MB 882.6 kB/s eta 0:00:16
   -- ------------------------------------- 1.0/14.9 MB 811.6 kB/s eta 0:00:18
   --- ------------------------------------ 1.3/14.9 MB 762.6 kB/s eta 0:00:18
   ---- ----------------------------------- 1.6/14.9 MB 830.6 kB/s eta 0:00:17
   ---- ----------------------------------- 1.6/14.9 MB 830.6 kB/s eta 0:00:17
   ---- ----------------------------------- 1.8/14.9 MB 825.2 kB/s eta 0:00:16
   ----- ---------------------------------- 2.1/14.9 MB 851.1 kB/s eta 0:00:16
   ----- ---------------------------------- 2.1/14.9 MB 851.1 kB/s eta 0:00:16


In [14]:
!nvidia-smi

Sat Nov  9 15:16:18 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.12                 Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A    0C    P8              1W /   60W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [21]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from langchain import hub
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

### INDEXING ###

# 1. Load Documents
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2024-07-07-hallucination/",), 
    bs_kwargs=dict(
        parse_only = bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

# 2. Split the documents/Chunking
## chunk first 1000 characters, then take next 1000 but overlap 200, eg: 1 - 1000, 800 - 1800 (We do this to reduce the error due to losing context )
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(documents=docs)

# 3. Embedding Documents.
## See if you can find a better model for embeddings  
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, 
    model_kwargs = model_kwargs, 
    encode_kwargs = encode_kwargs
)

vector_store = FAISS.from_documents(
    documents=splits, 
    embedding=hf_embeddings
)

# Taking Dense Retrieval - Embeddings/Context Based
retriever = vector_store.as_retriever()

### Retrieval & Generation ### 
prompt = hub.pull("rlm/rag-prompt")
# Since this wasnt working for me I checked the documentation and created my own function
# prompt = lang_prompt

llm = ChatGroq(
    model = "llama3-8b-8192", 
    temperature = 0
)

# post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | llm
    | StrOutputParser()
)

# Question 
print(rag_chain.invoke("What is in-context hallucination?"))
 

In-context hallucination refers to the model output being consistent with the source content in context, but not necessarily grounded in world knowledge. This type of hallucination is different from extrinsic hallucination, which is when the model output is not grounded in either the provided context or world knowledge.


In [20]:
docs[0].page_content.strip()

'Extrinsic Hallucinations in LLMs\n    \nDate: July 7, 2024  |  Estimated Reading Time: 30 min  |  Author: Lilian Weng\n\n\nHallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation. If we consider the pre-training data corpus as a proxy for world knowledge, we essentially try to ensure th